## k-nearest neighbor (kNN) search
* 질의에 사용된 벡터와 가까운 k개의 벡터를 탐색

### Prerequisites
* `dense_vector` 필드 값으로 저장된 임베딩 벡터
  * [Elasticsearch에서 제공하는 NLP 모델](https://www.elastic.co/guide/en/machine-learning/8.12/ml-nlp-text-emb-vector-search-example.html) 혹은 외부 모델로부터 생성
    * Elasticsearch에서 제공하는 NLP 모델을 사용하기 위해서는 유료 라이선스 필요 
* 인덱스에 대해 아래 권한
  * `dense_vector` 필드를 포함한 인덱스 생성을 위한 `create_index` 혹은 `manage` 권한
  * 데이터를 인덱스에 추가하기 위한 `create`, `index` 혹은 `write` 권한
  * 검색을 위한 `read` 권한 

In [ ]:
%%bash
# Elasticsearch에서 제공하는 NLP 모델 사용 방법
# 모델 생성
docker run -it --rm docker.elastic.co/eland/eland \
  eland_import_hub_model \
  --url http://localhost:9200/ \
  --hub-model-id sentence-transformers/msmarco-MiniLM-L-12-v3 \
  --task-type text_embedding \
  --start
  
# embedding
curl -X POST --location "http://localhost:9200/_ml/trained_models/sentence-transformers__msmarco-minilm-l-12-v3/_infer" \
    -H "Content-Type: application/json" \
    -d "{
          \"docs\": {
            \"text_field\": \"How is the weather in Jamaica?\"
          }
        }"

### kNN methods
* 엘라스틱서치에서는 아래 2가지 방법을 지원
  * Approximate kNN
    * 정확도와 색인 속도를 희생해서 검색 속도를 향상
    * `knn` 검색 옵션 혹은 `knn` 쿼리 사용
  * Exact, brute-force kNN
    * 모든 문서에 대해 벡터 함수 적용해 탐색
    * 큰 데이터셋에 부적절
    * 벡터 함수와 함께 `script_score` 쿼리 사용
    
### Approximate kNN
* 검색 순서
  1. 하나 이상의 `dense_vector` 필드를 mapping에 정의
    * 관련 옵션은 [Parameters for dense vector fields](https://www.elastic.co/guide/en/elasticsearch/reference/current/dense-vector.html#dense-vector-params) 페이지 참고
  2. 데이터 색인
  3. `knn` 검색 옵션 혹은 `knn` 쿼리 사용
* byte 값 vector 검색 지원
* quantization 지원
* pre-filtering / post-filtering 지원
* 다른 검색과 함께 검색 가능
* 여러 `dense vector` 필드에 대해 ann 검색 가능
* 튜닝 방법
  * `num_candidates` 값을 높게 설정할 수록 속도는 느려지지만 정확도는 올라감
    * `num_candidates` 값은 각 샤드에서 검색해올 벡터의 개수를 뜻함
  * quantization 적용
    * 384 차원 이상의 float 벡터의 경우 적용을 권장
  * PCA 등의 기법을 통해 벡터의 차원을 축소
  * `_source`에서 벡터 값 제외
  * `index.store.preload` 설정을 통한 파일 시스템 캐시 warm up
  * 세그먼트 수를 줄임
  * 검색 시 무거운 색인을 지양
  * readahead 값을 적정 수준으로 조정

In [1]:
%%bash
curl -X PUT "localhost:9200/image-index?pretty" -H 'Content-Type: application/json' -d'
{
  "mappings": {
    "properties": {
      "image-vector": {
        "type": "dense_vector",
        "dims": 3,
        "similarity": "l2_norm"
      },
      "title-vector": {
        "type": "dense_vector",
        "dims": 5,
        "similarity": "l2_norm"
      },
      "title": {
        "type": "text"
      },
      "file-type": {
        "type": "keyword"
      }
    }
  }
}
'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   474  100    87  100   387    365   1626 --:--:-- --:--:-- --:--:--  2106


{
  "acknowledged" : true,
  "shards_acknowledged" : true,
  "index" : "image-index"
}


In [3]:
%%bash
curl -X POST "localhost:9200/image-index/_bulk?refresh=true&pretty" -H 'Content-Type: application/json' -d'
{ "index": { "_id": "1" } }
{ "image-vector": [1, 5, -20], "title-vector": [12, 50, -10, 0, 1], "title": "moose family", "file-type": "jpg" }
{ "index": { "_id": "2" } }
{ "image-vector": [42, 8, -15], "title-vector": [25, 1, 4, -12, 2], "title": "alpine lake", "file-type": "png" }
{ "index": { "_id": "3" } }
{ "image-vector": [15, 11, 23], "title-vector": [1, 5, 25, 50, 20], "title": "full moon", "file-type": "jpg" }
'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1568  100  1145  100   423  29643  10951 --:--:-- --:--:-- --:--:-- 47515


{
  "errors" : false,
  "took" : 28,
  "items" : [
    {
      "index" : {
        "_index" : "image-index",
        "_id" : "1",
        "_version" : 1,
        "result" : "created",
        "forced_refresh" : true,
        "_shards" : {
          "total" : 2,
          "successful" : 1,
          "failed" : 0
        },
        "_seq_no" : 0,
        "_primary_term" : 1,
        "status" : 201
      }
    },
    {
      "index" : {
        "_index" : "image-index",
        "_id" : "2",
        "_version" : 1,
        "result" : "created",
        "forced_refresh" : true,
        "_shards" : {
          "total" : 2,
          "successful" : 1,
          "failed" : 0
        },
        "_seq_no" : 1,
        "_primary_term" : 1,
        "status" : 201
      }
    },
    {
      "index" : {
        "_index" : "image-index",
        "_id" : "3",
        "_version" : 1,
        "result" : "created",
        "forced_refresh" : true,
        "_shards" : {
          "total" : 2,
          "s

In [4]:
%%bash
curl -X POST "localhost:9200/image-index/_search?pretty" -H 'Content-Type: application/json' -d'
{
  "knn": {
    "field": "image-vector",
    "query_vector": [-5, 9, -12],
    "k": 10,
    "num_candidates": 100
  },
  "fields": [ "title", "file-type" ]
}
'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2106    0  1946  100   160  38203   3141 --:--:-- --:--:-- --:--:-- 51365


{
  "took" : 29,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 3,
      "relation" : "eq"
    },
    "max_score" : 0.008547009,
    "hits" : [
      {
        "_index" : "image-index",
        "_id" : "1",
        "_score" : 0.008547009,
        "_source" : {
          "image-vector" : [
            1,
            5,
            -20
          ],
          "title-vector" : [
            12,
            50,
            -10,
            0,
            1
          ],
          "title" : "moose family",
          "file-type" : "jpg"
        },
        "fields" : {
          "file-type" : [
            "jpg"
          ],
          "title" : [
            "moose family"
          ]
        }
      },
      {
        "_index" : "image-index",
        "_id" : "3",
        "_score" : 6.1349693E-4,
        "_source" : {
          "image-vector" : [
            15,
            11,


## 참고 자료
* [Introducing approximate nearest neighbor search in Elasticsearch 8.0](https://www.elastic.co/kr/blog/introducing-approximate-nearest-neighbor-search-in-elasticsearch-8-0)
* [k-nearest neighbor (kNN) search](https://www.elastic.co/guide/en/elasticsearch/reference/current/knn-search.html#knn-search)
* [Knn query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-knn-query.html)
* [Tune approximate kNN search](https://www.elastic.co/guide/en/elasticsearch/reference/current/tune-knn-search.html)